In [1]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav

In [2]:
# Settings
duration = 3   # seconds
fs = 16000     # sample rate (16 kHz)

In [5]:
print("🎤 Recording... Speak now!")
recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
sd.wait()  # wait until done
print("✅ Recording complete!")

# Save as a file (optional)
wav.write("Recordings/whispy_test.wav", fs, recording)

# Playback
print("🔊 Playing back your voice...")
sd.play(recording, fs)
sd.wait()
print("🎉 Done! That’s your voice.")


🎤 Recording... Speak now!
✅ Recording complete!
🔊 Playing back your voice...
🎉 Done! That’s your voice.
